In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
train=pd.read_csv('../input/nlp-getting-started/train.csv')
test=pd.read_csv('../input/nlp-getting-started/test.csv')
train

In [3]:
train.isna().sum()

In [4]:
train['keyword'].unique().shape


In [5]:
train['keyword'].value_counts().head(10).plot(kind='bar')

In [6]:
#sklean countvectorizer
from sklearn.feature_extraction.text import CountVectorizer
bow=CountVectorizer(max_features=1000)
train_bow=bow.fit_transform(train['text'])
test_bow=bow.transform(test['text'])



In [7]:
#fill nan keyword with 0
train['keyword'].fillna('0',inplace=True)
test['keyword'].fillna('0',inplace=True)

#assign int to keyword
from sklearn.preprocessing import LabelEncoder
label=LabelEncoder()
train['keyword_label']=label.fit_transform(train['keyword'])
test['keyword_label']=label.transform(test['keyword'])
#X- train_bow+ train['keyword_label']


In [8]:
keywords_train=train['keyword'].values
#concatenate


In [9]:
X_train=train_bow

In [10]:
#split train and test
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(X_train,train['target'],test_size=0.2,random_state=42)
#

In [11]:
#logistic regression
from sklearn.linear_model import LogisticRegression
logreg=LogisticRegression(max_iter=10000)
logreg.fit(x_train,y_train)
#score
logreg.score(x_test,y_test)

In [12]:
logreg.score(x_train,y_train)

In [13]:
#prediction on test
pred=logreg.predict(test_bow)
#save prediction
pd.DataFrame({'id':test['id'],'target':pred}).to_csv('submission.csv',index=False)



In [14]:
!pip install tensorflow_text


In [15]:
#bert
import tensorflow_text as text
import tensorflow as tf
import tensorflow_hub as hub
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string)
preprocessor = hub.KerasLayer(
    "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
encoder_inputs = preprocessor(text_input)
encoder = hub.KerasLayer(
    "https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-256_A-4/2",
    trainable=False,)
outputs = encoder(encoder_inputs)
pooled_output = outputs["pooled_output"]      # [batch_size, 256].
sequence_output = outputs["sequence_output"]  # [batch_size, seq_length, 256].
#dense layer
dense=tf.keras.layers.Dense(1,activation='sigmoid')(pooled_output)
model=tf.keras.Model(inputs=text_input,outputs=dense)
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

In [16]:
reduce_lr=tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss',factor=0.5,patience=5)
early_stopping=tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=10)
callbacks=[reduce_lr,early_stopping]


In [17]:
import tensorflow as tf

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


In [ ]:
model.fit(train['text'],train['target'],batch_size=1024,epochs=10,validation_split=0.2,callbacks=callbacks)

In [ ]:
from transformers import DistilBertTokenizer, TFDistilBertForSequenceClassification 

tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased",num_labels=2)
model = TFDistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased",num_labels=2)



In [ ]:
#tokenize
train_tokens = tokenizer(train.text.values.tolist(), return_tensors="tf", padding=True, truncation=True, max_length=512)
test_tokens = tokenizer(test.text.values.tolist(), return_tensors="tf", padding=True, truncation=True, max_length=512)

In [ ]:
#select attention_mask and input_ids and token_type_ids
train_input_ids = train_tokens["input_ids"]
train_attention_mask = train_tokens["attention_mask"]
test_input_ids = test_tokens["input_ids"]
test_attention_mask = test_tokens["attention_mask"]
tr_token=tf.concat([train_input_ids,train_attention_mask],axis=1)
te_token=tf.concat([test_input_ids,test_attention_mask],axis=1)

In [ ]:
train_labels=train.target.values
train_data=tf.data.Dataset.from_tensor_slices((tr_token,train_labels)).shuffle(len(train_labels)).batch(64)


In [ ]:
loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model = TFDistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased",num_labels=2)

optimizer=tf.keras.optimizers.Adam(learning_rate=6e-4)
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
model.fit(train_data,batch_size=64, epochs=5)

In [ ]:
from transformers import AutoModel, AutoTokenizer,TFAutoModelForSequenceClassification ,AutoConfig


config = AutoConfig.from_pretrained("vinai/bertweet-base",num_labels=2)
berttweet = TFAutoModelForSequenceClassification.from_config(config)
tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-base",num_labels=2)




In [ ]:
train_tokens = tokenizer(train.text.values.tolist(), return_tensors="tf", padding=True, truncation=True, max_length=512)
test_tokens = tokenizer(test.text.values.tolist(), return_tensors="tf", padding=True, truncation=True, max_length=512)
#select attention_mask and input_ids and token_type_ids
train_input_ids = train_tokens["input_ids"]
train_attention_mask = train_tokens["attention_mask"]
test_input_ids = test_tokens["input_ids"]
test_attention_mask = test_tokens["attention_mask"]
tr_token=tf.concat([train_input_ids,train_attention_mask],axis=1)
te_token=tf.concat([test_input_ids,test_attention_mask],axis=1)
train_data=tf.data.Dataset.from_tensor_slices((tr_token,train_labels)).shuffle(len(train_labels)).batch(32)
    

In [ ]:
loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optimizer=tf.keras.optimizers.Adam(learning_rate=6e-4)
berttweet.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
berttweet.fit(train_data,batch_size=32, epochs=5)